In [13]:
import networkx as nx
import pandas as pd
import math
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
from abc import ABC, abstractmethod
import numpy as np


In [14]:
def V(num):
    return 4*num+1

def S(num):
    return 2*num+1

def fG(num):
    return 2*num-1

def getType(num):
    if (num+1) %3 == 0:
        return "A"
    elif (num%3) == 0:
        return "B"
    elif (num-1)%3 == 0:
        return "C"

In [15]:
class Tree:
    
    def __init__(self, num_rows):
        self.num_rows = num_rows
        self.rows = []
        self.generate()

    def generate(self):
        for i in range(self.num_rows):
            row = []
            for j in range(2**i, 2**(i+1)):
                if j % 2 is 1:
                    node = Node(j)
                    row.append(node)
            self.rows.append(row)
    
    def getNode(self, label):
        row = Node.getRow(label)
        return self.rows[row][(label-(2**row+1))//2]

    def mark(self, label):
        self.getNode(label).marked = True

    def checkRowMarked(self) -> int:
        for i, row in enumerate(self.rows):
            rowIsMarked = True
            for node in row:
                if node.marked == False:
                    rowIsMarked = False
                    break
            if rowIsMarked == True:
                return i
        return -1
    
    def countMarkedNodesAboveRow(self, row) -> int:
        cnt = 0
        for row in self.rows[row+1:]:
            for node in row:
                if node.marked == True:
                    cnt += 1
        return cnt

class Node:
    def __init__(self, label):
        self.tree = None
        self.row = Node.getRow(label)
        self.marked = False
        self.label = label
    
    @staticmethod
    def getRow(label: int) -> int:
        return int(math.log(label,2))

    def setTree(self, tree: Tree):
        self.tree = tree

    @staticmethod
    def getType(label):
        if (label + 1) % 3 == 0:
            if ((label-1)/4)%2 == 0:
                return 4
            return 1
        elif label % 3 == 0:
            return 2
        elif (label - 1) % 3 == 0:
            return 3
        return -1

In [16]:
def getCol(height):    

    maxValueAllowed = 2**height-1

    numArr = []
    bigLeaf = 4096

    xMax1 = bigLeaf
    x1 = 1

    while x1 < xMax1:
        y1 = V(x1)
        if x1 < maxValueAllowed and y1 < maxValueAllowed:
            numArr.append([x1,y1,getType(y1)])   

        x1 = x1+1



    #print(1)



    xMax2 = 300
    kMax2 = 10

    for x2 in range(1,xMax2+1):
        #if V(x) > big_leaf:
        #        break
        for k2 in range(1,kMax2+1):
            
            

            #for i in range(1,k+1):
            firstPartSave2 = V(x2)
            secondPartSave2 = V(x2)

            for j in range(0,k2):
                firstPartSave2 = S(firstPartSave2)
            
            for m in range(0,k2+1):
                secondPartSave2 = S(secondPartSave2)



            firstPart2 = firstPartSave2
            secondPart2 = secondPartSave2
            firstType2 = getType(firstPart2)
            secondType2 = getType(secondPart2)


            if x2 < maxValueAllowed and firstPart2 < maxValueAllowed:
                numArr.append([x2, firstPart2, firstType2])
            if x2 < maxValueAllowed and secondPart2 < maxValueAllowed:
                numArr.append([x2, secondPart2, secondType2])



    #print(2)

    xMax4 = 300
    nMax4 = 10

    for n4 in range(1,nMax4+1):

        for x4 in range(1,xMax4+1):
            if x4%3 == 0:
                True
            else:
                y4 = S(3**n4 * x4)

                for i4 in range(1,n4+1):
                    firstPart4 = S((4**i4)*(3**(n4-i4)*x4))
                    secondPart4 = S(firstPart4*x4)
                    firstType4 = getType(firstPart4)
                    secondType4 = getType(secondPart4)

                    if y4 < maxValueAllowed and firstPart4 < maxValueAllowed:
                        numArr.append([y4, firstPart4, firstType4])
                    if y4 < maxValueAllowed and secondPart4 < maxValueAllowed:
                        numArr.append([y4, secondPart4, secondType4])


    #print(4)


    xMax3 = 300
    nMax3 = 10
    for n3 in range(1,nMax3+1):

    #n = 4

        for x3 in range(1,xMax3+1):
            if x3%3 == 0:
                True
            else:
                y3 = 3**n3 * x3

                for i3 in range(1,n3+1):
                    firstPart3 = V((4**i3)*(3**(n3-i3)*x3))
                    secondPart3 = S(firstPart3*x3)
                    firstType3 = getType(firstPart3)
                    secondType3 = getType(secondPart3)

                    if y3 < maxValueAllowed and firstPart3 < maxValueAllowed:
                        numArr.append([y3, firstPart3, firstType3])
                    if y3 < maxValueAllowed and secondPart3 < maxValueAllowed:
                        numArr.append([y3, secondPart3, secondType3])

    #print(3)


    xMax5 = 300
    nMax5 = 10

    for n5 in range(1,nMax5+1):

        for x5 in range(1,xMax5+1):
            if x5%3 == 0:
                True
            else:
                y5 = fG(3**n5 * x5)

                if (y5+1)%3 == 0:

                    for i5 in range(1,n5+1):
                        firstPartSave5 = fG((x5))
                        secondPartSave5 = fG((x5))

                        for j in range(0,i5):
                            firstPartSave5 = S(firstPartSave5)
                        
                        for m in range(0,i5+1):
                            secondPartSave5 = S(secondPartSave5)



                        firstPart5 = firstPartSave5
                        secondPart5 = secondPartSave5
                        firstType5 = getType(firstPart5)
                        secondType5 = getType(secondPart5)

                        if y5 < maxValueAllowed and firstPart5 < maxValueAllowed:
                            numArr.append([y5, firstPart5, firstType5])
                        if y5 < maxValueAllowed and secondPart5 < maxValueAllowed:
                            numArr.append([y5, secondPart5, secondType5])

    #print(5)

    start = 31
    end = 17
    treeNumbers = []
    maxRec = 15

    numArr.sort()
    length = len(numArr)-1
    for i in range(0, length):
        if numArr[length-i] == numArr[length-i-1]:
            numArr.pop(length-i)

    #print(numArr)

    startKnots = dict()
    pre = []
    for i in numArr:

        if pre == []:
            pre = i
            key = i[0]
            

            startKnots[key] = [i[1]]
        else:
            if i[0]==pre[0]:
                key = pre[0]
                startKnots[key].append(i[1])
            else:
                key = i[0]
                startKnots[key] = [i[1]]
                pre = i


    return startKnots

In [17]:
tree = Tree(5)

numCol = getCol(5)
#the higher the n is, the the higher the x,n and k values have to be chosen in the rule function to prove all values
#print(numCol)
for i in numCol:
    #print(i)
    for j in numCol[i]:
        tree.mark(j)
        markedRow = tree.checkRowMarked()
        
        if markedRow > 0:
            tree.countMarkedNodesAboveRow(markedRow)

#tree.mark(5)
#tree.mark(7)

#tree.mark(11)
#tree.mark(19)
#tree.mark(31)

for row in tree.rows:
    for node in row:
        print("{0}-{1}".format(node.label, 'T' if node.marked else 'F'), end =" ")
    print('\n')

markedRow = tree.checkRowMarked()
print(markedRow)
print(tree.countMarkedNodesAboveRow(markedRow))


1-F 

3-T 

5-T 7-T 

9-T 11-T 13-T 15-T 

17-T 19-T 21-T 23-T 25-T 27-T 29-T 31-T 

33-T 35-T 37-T 39-T 41-T 43-T 45-T 47-T 49-T 51-T 53-T 55-T 57-T 59-T 61-T 63-T 

65-T 67-T 69-T 71-T 73-T 75-T 77-T 79-T 81-T 83-T 85-T 87-T 89-T 91-T 93-T 95-T 97-T 99-T 101-T 103-T 105-T 107-T 109-T 111-T 113-T 115-T 117-T 119-T 121-T 123-T 125-T 127-T 

129-T 131-T 133-T 135-T 137-T 139-T 141-T 143-T 145-T 147-T 149-T 151-T 153-T 155-T 157-T 159-T 161-T 163-T 165-T 167-T 169-T 171-T 173-T 175-T 177-T 179-T 181-T 183-T 185-T 187-T 189-T 191-T 193-T 195-T 197-T 199-T 201-T 203-T 205-T 207-T 209-T 211-T 213-T 215-T 217-T 219-T 221-T 223-T 225-T 227-T 229-T 231-T 233-T 235-T 237-T 239-T 241-T 243-T 245-T 247-T 249-T 251-T 253-T 255-T 

257-T 259-T 261-T 263-T 265-T 267-T 269-T 271-T 273-T 275-T 277-T 279-T 281-T 283-T 285-T 287-T 289-T 291-T 293-T 295-T 297-T 299-T 301-T 303-T 305-T 307-T 309-T 311-T 313-T 315-T 317-T 319-T 321-T 323-T 325-T 327-T 329-T 331-T 333-T 335-T 337-T 339-T 341-T 343-T 345-T 3